In [1]:
import pennylane as qml
from pennylane import numpy as np
from Ising_Inspired_QLSP import *

In [12]:
n = 4
qubit_wires = range(n)
dim = 2**n
kappa = 60
layers = 4

gate_set, coefficient_set, A = Ising_Inspired_QLSP(n, kappa=kappa, algorithm='shadows')
b = [1/np.sqrt(dim) for i in range(dim)]

In [13]:
#single layer
def layer(qubit_wires, parameters):
  n = len(qubit_wires)
  if n%2==0:
    qml.broadcast(qml.CNOT,wires=qubit_wires, pattern='double')
    qml.broadcast(qml.RY, wires=qubit_wires, parameters=parameters[0:n], pattern='single')
    qml.broadcast(qml.CNOT,wires=qubit_wires[1:], pattern='double')
    qml.broadcast(qml.RY, wires=qubit_wires[1:-1], parameters=parameters[n:2*(n-1)], pattern='single')
  else:
    qml.broadcast(qml.CNOT,wires=qubit_wires, pattern='double')
    qml.CNOT(wires=[qubit_wires[0], qubit_wires[-1]])
    qml.broadcast(qml.RY, wires=qubit_wires, parameters=parameters[0:n], pattern='single')
    qml.broadcast(qml.CNOT,wires=qubit_wires[1:], pattern='double')
    qml.CNOT(wires=[qubit_wires[0], qubit_wires[-1]])
    qml.broadcast(qml.RY, wires=qubit_wires, parameters=parameters[n:2*n], pattern='single')

In [14]:
#Fixed anstaz
def apply_fixed_ansatz(qubit_wires, parameters, layers):  
  n = len(qubit_wires)  
  qml.broadcast(qml.RY, wires=qubit_wires, parameters=parameters[0:n], pattern='single')
  
  if n%2==0:
    for l in range(layers):
      layer(qubit_wires, parameters[n+2*(n-1)*l:n+2*(n-1)*l+2*(n-1)])
  else:
    for l in range(layers):
      layer(qubit_wires, parameters[n+2*n*l:n+2*n*l+2*n])

In [15]:
dev2 = qml.device('lightning.qubit', wires=qubit_wires)

@qml.qnode(dev2)
def circuit_fielity(parameters):
    apply_fixed_ansatz(qubit_wires, parameters, layers)
    return qml.state()

In [16]:
def calc_fidelity(state_vec, A, b):
    Ainv = np.linalg.inv(A)
    Ainvprodb = np.matmul(Ainv,b)
    Ainvprodb_norm = Ainvprodb/np.linalg.norm(Ainvprodb)  
    fidelity = np.dot(np.array(Ainvprodb_norm), state_vec)
    return fidelity

In [21]:
for i in range(1,11):
    fname = f'Results_PI/Ising_4qubits_0.01serr_params_{i}seed_Powell.npy'
    params = np.load(fname)
    state_vec = circuit_fielity(params)
    print(calc_fidelity(state_vec, A, b))

(0.9956338877525633+0j)
(0.9979254325935571+0j)
(0.9964425981442757+0j)
(0.9934536364544231+0j)
(0.9941422039741235+0j)
(0.9937009982681992+0j)
(0.9947147949652606+0j)
(0.9958393907383181+0j)
(0.9980484775387102+0j)
(0.9979524257504259+0j)
